# Countdown (Numbers round)

*By Chayapol "Due" Hongsrimuang (G00388741)*

---

# Introduction
*Countdown* is a television programme in Channel 4. It is based on a French version of the programme *Des chiffres et das lettres* (translated to Numbers and letters) and has been running since November 2, 1982, when the channel is launched.

One of the rounds that are played in the programme is the **Numbers round**, where contestants are given 6 random numbers and have to use them to try and reach a randomly-generated target. (from 101-999, inclusive)

In [16]:
# Generating the target
import math, random

target = math.floor(random.random() * 900) + 100
target

843

The 6 random numbers are selected by the contestants from a pool of small and large numbers. Contestants can choose from 2 sets of small numbers (1-10) and a set of large numbers. (25, 50, 75, 100)

In [17]:
# Declaring the pool of numbers that can be selected
small_numbers = [number for number in range(1, 11)]     # 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
big_numbers = [number * 25 for number in range(1, 5)]   # 25, 50, 75, 100 (multiples of 4)

numbers_pool = small_numbers + small_numbers + big_numbers
numbers_pool

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 25, 50, 75, 100]

In [18]:
# Generating 6 random numbers from the pool
import math, random

def get_numbers(pool):
    num_pool = pool.copy()
    chosen_numbers = []
    for i in range(1, 7):
        random_index = math.floor(random.random() * len(num_pool))  # Get the index of random number to be chosen
        chosen_numbers.append(num_pool[random_index])               # Append that number to the list
        num_pool.remove(num_pool[random_index])                     # Remove the number from the pool
    return chosen_numbers

numbers = get_numbers(numbers_pool)
numbers

[7, 9, 4, 25, 1, 5]

With those numbers, contestants would be given 30 seconds to use those numbers and try to reach the target. They are only allowed to use basic mathematical operations (addition, subtraction, multiplication, and division) and no fractions are allowed in the case of divisions.

# Algorithm

## Step 1: Numbers Permutation

Firstly, as a brute force method, we would have to declare the permutations for the numbers pool. There are a total of 720 possible permutations for a number pool of 6. The permutation amount is determined by the amount of numbers in the pool, where it increase factorially. The equation derived from this is:
$$ n! $$
where n is the amount of numbers in the numbers pool.

However, with the addition of operators to each permutation, this number (720) will increase.

In [19]:
# Get permutations from values
from itertools import permutations

def get_permutations(arr):
    return list(permutations(numbers, 6))

perms = get_permutations(numbers)
len(perms)

720

## Step 2: Operators Permutation
Next, the operators are added in between the numbers of index 2 onwards. Each permutation is transformed in a form of a reverse Polish notation.

This is where a math expression is written in the form like this:

$$ n \; n \; o \; n \; o ... $$

where $ n $ is the number and $ o $ is the operator.

This notation acts as a "queue" of elements to be added to a stack. The way this will function is later explained in the next section.

Otherwise, for each permutation from before, it will generate 1,024 further solutions. When multiplied with the earlier permutation count, the number goes to 737,280.
This means that there are possible 737,280 solutions that can be derived from a pool of numbers and the operators added into.

In [20]:
import operator # from https://www.ttested.com/polish-countdown/

# Operations - from https://www.ttested.com/polish-countdown/
operators = [operator.add, operator.sub, operator.mul, operator.floordiv]

# Add an operator for the current stack
def add_an_operator(num_arr):
    ret = []
    for i in range(len(operators)):
        ret.append(num_arr + [operators[i]])

    return ret

def add_operators(num_arr):
    left = num_arr.copy()
    stack = []

    # Initialise the initial stack
    for i in range(0, 2):
        stack.append(left.pop())

    prev = [stack]

    current_op = len(left) + 1
    while current_op != 0: # while there is still un-used numbers
        # Initial location for new stacks
        modified_arrays = []

        # If the previous added item is an operator, add the number instead
        first_stack = prev[0]
        if first_stack[-1] in operators:
            number_to_add = left.pop()
            # Add the number popped, pushed the change for each stack into modified_arrays
            for a_stack in prev:
                a_stack.append(number_to_add)
                modified_arrays.append(a_stack)
            prev = modified_arrays # assign previous as the current modified_arrays
            current_op = current_op - 1
            continue # continue with the next iteration

        # Elsewise, if the previous added item is not an operator, add the operators    
        for a_stack in prev:
            # Add operators, return 4 arrays
            four_arrays = add_an_operator(a_stack)

            # Add the four arrays to the return array 
            for each_array in four_arrays:
                #print(each_array)
                modified_arrays.append(each_array)

        if current_op == 1:
            current_op = 0 # Break the loop if final operation done

        prev = modified_arrays # assign previous as the current modified_arrays
            
    return prev

example_perm = perms[0]
possible_work = add_operators(list(example_perm).copy())
print(possible_work) # Possible work from a permutation
print(len(possible_work)) # length of that possible work

# Possible permutations with added operators
print(str(len(possible_work) * len(perms)))

[[5, 1, <built-in function add>, 25, <built-in function add>, 4, <built-in function add>, 9, <built-in function add>, 7, <built-in function add>], [5, 1, <built-in function add>, 25, <built-in function add>, 4, <built-in function add>, 9, <built-in function add>, 7, <built-in function sub>], [5, 1, <built-in function add>, 25, <built-in function add>, 4, <built-in function add>, 9, <built-in function add>, 7, <built-in function mul>], [5, 1, <built-in function add>, 25, <built-in function add>, 4, <built-in function add>, 9, <built-in function add>, 7, <built-in function floordiv>], [5, 1, <built-in function add>, 25, <built-in function add>, 4, <built-in function add>, 9, <built-in function sub>, 7, <built-in function add>], [5, 1, <built-in function add>, 25, <built-in function add>, 4, <built-in function add>, 9, <built-in function sub>, 7, <built-in function sub>], [5, 1, <built-in function add>, 25, <built-in function add>, 4, <built-in function add>, 9, <built-in function sub>, 7

# Step 3: Stack Operations
For each permutation, this acts as a queue, for each element to be pushed into a stack. A stack, in this case, consists of at most two numbers, and will be evaluated once an operator is pushed into the stack.

Here's an example of how this works. Let's say that we have a queue of $ q = \{5, 1, +, 75 , +, 6, +\} $.
* Firstly, the two numbers are pushed into a stack (s) $$ q = \{+, 75, +, 6, + \} $$ $$ s = \{5, 1 \} $$
* Then, the operator (+) is pushed into the stack $$ q = \{75, +, 6, + \} $$ $$ s = \{5, 1, + \} $$
* With that operator, this is then used for the two numbers in the stack. The result is pushed into the stack. $$ q = \{75, +, 6, + \} $$ $$ s = \{6\} $$
* After which, the process repeats until the queue is empty, and stack consists of only one number, which is the final result.

However, the queue has to finish with an operator, in order for the operations on the notation to be complete. If the queue finishes with a number, there would be no finish result from the queues, as it exists two numbers in the stack.

These two functions act upon each permutation that is described from both the numbers permutation and the stack permutation.

In [21]:
# Operate on the permutation, acting as a queue
def operate(arr):
    res = operate_helper(arr.pop(0), arr.pop(0), arr.pop(0))

    while len(arr) != 0:
        the_number = arr.pop(0)
        the_operator = arr.pop(0)

        res = operate_helper(res, the_number, the_operator)
    
    return res

# do the equation
def operate_helper(num1, num2, op):
    res = op(num1, num2)
    return res

operate(possible_work[0].copy())


51

Finally, the `solve_numbers()` function is declared, consisting of all the functionalities above:
* First, all the possible permutations are declared
* For each permutation, get all the combinations with all operators.
    * For each combination, evaluate them via reverse polish notation.
        * If the combination produces a target number, return the solution, indicating that a solution is found.
        * If the combination doesn't produce a target number, continue.
* Else, if all permutations do not produce the target number, return "No solution found".

In [24]:
def solve_numbers(num_array, target):
    # Get permutations
    perms = get_permutations(num_array)

    # For each permutation
    for perm in perms:
        # Change to be mutable
        perm = list(perm)
        
        # Get all the combinations with all operators
        perm_with_ops = add_operators(perm)

        # For each combinations, run down the stack
        for perm_ops in perm_with_ops:
            res = operate(perm_ops.copy())
            if res == target:
                print("SOLUTION FOUND FOR "+str(target))
                print(perm_ops)
                return

    # If no solutions can be found, print "No solution"
    print("No solution for "+str(target))
    return

solve_numbers(numbers, target)

SOLUTION FOUND FOR 843
[1, 4, <built-in function add>, 5, <built-in function mul>, 9, <built-in function add>, 25, <built-in function mul>, 7, <built-in function sub>]


---
End